In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely
import shapefile
import flopy
from flopy.utils.gridgen import Gridgen
from flopy.discretization import StructuredGrid, VertexGrid
from flopy.utils.triangle import Triangle as Triangle
from flopy.utils.voronoi import VoronoiGrid
from flopy.utils.gridintersect import GridIntersect

In [ ]:
model_ws = "temp/structured/"

Load a few raster files

In [ ]:
bottom = flopy.utils.Raster.load("data_project/aquifer_bottom.asc")
top = flopy.utils.Raster.load("data_project/aquifer_top.asc")
kaq = flopy.utils.Raster.load("data_project/aquifer_k.asc")

Load a few shapefiles with geopandas

In [ ]:
river = gpd.read_file("data_project/Flowline_river.shp")
inactive = gpd.read_file("data_project/inactive_area.shp")
active = gpd.read_file("data_project/active_area.shp")
wells = gpd.read_file("data_project/pumping_well_locations.shp")

Plot the shapefiles with geopandas

In [ ]:
ax = river.plot(color="cyan")
active.plot(ax=ax, color="blue")
inactive.plot(ax=ax, color="white")
wells.plot(ax=ax, color="red", markersize=8);

#### Make a structured model grid

In [ ]:
nlay, nrow, ncol = 3, 40, 20  # 80, 40
shape2d, shape3d = (nrow, ncol), (nlay, nrow, ncol)
xlen, ylen = 5000.0, 10000.0
delr = np.full(ncol, xlen / ncol, dtype=float)
delc = np.full(nrow, ylen / nrow, dtype=float)
ttop = np.full(shape2d, 1.0, dtype=float)
tbotm = np.full(shape3d, 0.0, dtype=float)

In [ ]:
# create base_grid using StructuredGrid() method


### Intersect the modelgrid with the shapefiles

#### Create an intersection object

In [ ]:
ix = GridIntersect(base_grid, method="vertex", rtree=True)

#### Intersect inactive and active shapefiles with the modelgrid

After all of the intersection operations, take a look at the data contained in the returned objects

#### Intersect well shapefile with the modelgrid

In [ ]:
well_cells = []
for g in wells.geometry:
    v = ix.intersect(g)
    well_cells += v["cellids"].tolist()

In [ ]:
well_cells

#### Intersect river shapefile with the modelgrid

In [ ]:
river_cells = ix.intersect(river.geometry[0])

In [ ]:
river_cells[:4], river_cells.dtype

### Intersect constant head line with the modelgrid

Use a line with two points to defined the location of the constant head cells. The line verticase are `[(1250, 0.1), (4250, 0.1)]`.

In [ ]:
constant_cells = ix.intersect(
    [(1250, 0.1), (4250, 0.1)], shapetype="linestring"
)

In [ ]:
constant_cells[:4], constant_cells.dtype

### Resample the raster data to the modelgrid

Use the `resample_to_grid()` method on each raster object.

In [ ]:
rtop = top.resample_to_grid(
    base_grid,
    band=top.bands[0],
    method="linear",
    extrapolate_edges=True,
)
rbot = bottom.resample_to_grid(
    base_grid,
    band=bottom.bands[0],
    method="linear",
    extrapolate_edges=True,
)
rkaq = (
    kaq.resample_to_grid(
        base_grid, band=kaq.bands[0], method="linear", extrapolate_edges=True
    )
    * 86400.0
)

### Plot the resampled data 

Plot the aquifer top, bottom, and hydraulic conductivity. Also plot the aquifer thickness.

#### Build the model data

_Create the bottom of each model layer_

Assume that the thickness of each layer at a row, column location is equal.

_Create the idomain array_

Use the intersection data from the active and inactive shapefiles to create the idomain array

_Build the well package stress period data_

* The pumping rates are in the `wells` geopandas dataframe
* Pumping rates are in m/sec
* The wells are located in model layer 3

_Build the river package stress period data_

* Calculate the length of the river using the `"lengths"` key. 
* The vertical hydraulic conductivity of the river bed sediments is 3.5 m/d.
* The thickness of river bottom sediments at the upstream (North) and downstream (South) end of the river is 0.5 and 1.5 meters, respectively. 
* The river bottom at the upstream and downstream end of the river is 16.5 and 14.5 meters, respectively. The river width at the upstream and downstream end of the river is 5.0 and 10.0 meters, respectively. 
* The river stage at the upstream and downstream end of the river is 16.6 and 15.5 meters, respectively.
* Use the boundname `upstream` for river cells where the upstream end of the river cell is less than 5000 m from the North end of the model. Use the boundname `downstream` for all other river cells. 
* Make sure the river cell is in the upper most layer where the river bottom exceeds the bottom of the cell.

Use the upstream and downstream values to interpolate the river sediment thickness, bottom, width, and stage for each river cell.

The river cells will be connected to model layer 1. The river bottom, width, and stage values should be calculated at the center of the river reach.

In [ ]:
# define river data

In [ ]:
# calculate total river length


In [ ]:
# calculate river slope


In [ ]:
# calculate river bottom slope

In [ ]:
# calculate river width slope


In [ ]:
# calculate river stage slope


In [ ]:
# determine the boundname for each reach

In [ ]:
# create the river stress period data


_Define river observations_

In [ ]:
riv_obs = {
    "riv_obs.csv": [
        ("UPSTREAM", "RIV", "UPSTREAM"),
        ("DOWNSTREAM", "RIV", "DOWNSTREAM"),
    ],
}

_Define the constant head cells_

Assume the constant head cells are located in all three layers and have values equal to the downstream river stage (`stage_down`). Make sure the constant head stage is greater that the bottom of the layer.

_Define recharge rates_

* The recharge rate is 0.16000000E-08 m/sec

#### Build the model

Build a steady-state model using the data that you have created. Packages to create:

* Simulation
* TDIS (1 stress period, `TIME_UNITS='days'`)
* IMS (default parameters)

* GWF model (`SAVE_FLOWS=True`)
* DIS (`LENGTH_UNITS='meters'`)
* IC (`STRT=40.`)
* NPF (Unconfined, same K for all layers, `SAVE_SPECIFIC_DISCHARGE=True`, `icelltype=1`)
* RCH (array based)
* RIV (`BOUNDNAMES=True`,add `RIV` observations for defined boundnames)
* CHD
* WEL
* OC (Save `HEAD ALL` and `BUDGET ALL`)


In [ ]:
name = "project"

In [ ]:
# create the simulation file

In [ ]:
# create the TDIS file

In [ ]:
# create the IMS file

In [ ]:
# create the GWF model

In [ ]:
# create the DIS file

In [ ]:
# create the IC file

In [ ]:
# create the NPF file

In [ ]:
# create the RCH file

In [ ]:
# create the RIV file (riv =)

In [ ]:
# add the river observation file
riv.obs.initialize(
    filename=f"{name}.riv.obs",
    digits=9,
    print_input=True,
    continuous=riv_obs,
)

In [ ]:
# create the WEL file

In [ ]:
# create the CHD file

In [ ]:
# create the OC file

#### Write the model files

#### Run the model

#### Post-process the results

Use `gwf.output.` method to get the observations.

Use `gwf.output.` method to get the heads and specific discharge. Make a map and cross-sections of the data using `flopy.plot` methods. Plot specific discharge vectors on the map and cross-sections.

In [ ]:
# process the specific discharge (DATA-SPDIS) into qx, qy, qz using flopy.utils.postprocessing.get_specific_discharge


In [ ]:
# plot the results